# Linear regression

$D = (X, Y) = \{(x_i, y_i)\}_{i=1}^N$ - обучающая выборка, N - число объектов в выборке  
$X = \{x_{ij}\}$ - матрица признаков (design matrix), $X \in R^{Nxd}$,  
где d - размерность признаков.  
$Y = [y_1, ..., y_N]^T$ - целевой признак, значение которого необходимо научится предсказывать.  
В случае регрессии $Y \in R^N$


Определим ошибку на одном объекте выборки (loss): $Loss(y_i, \hat{y_i}) = (y_i - \hat{y_i})^2$    
Определим ожидаемую ошибку на всей выборке (в нашем случае это будет - MSE, Mean Square Error): $Q(f, x, y) = \frac{1}{N} \sum_{i=1}^N Loss(y_i, f(x_i))$  

где f(x) - наша модель регрессии  

Рассмотрим модель линейной регрессии: $f(x_i) = f(x_i; w) = \sum_{j=1}^d w_j x_{ij} + w_0 = W^TX + w_0$,  
где $W = [w_1, ..., w_d]^T$ - вектор весов, который нужно обучить.  

Тогда ожидаемую ошибку на всей выборке можно записать как:  
$Q(f, x, y) = \frac{1}{N} \sum_{i=1}^N Loss(y_i, f(x_i)) = \frac{1}{N} \sum_{i=1}^N Loss(y_i, \sum_{j=1}^d w_j x_{ij} + w_0) = \frac{1}{N} \sum_{i=1}^N (y_i - \sum_{j=1}^d w_j x_{ij} - w_0)^2$  


Для обучения нашей модели нам необходимо минизировать Q относительно W при заданных X,Y:  

$Q(w) = \frac{1}{N} \sum_{i=1}^N (y_i - \sum_{j=1}^d w_j x_{ij} - w_0)^2 \rightarrow \underset{W}{min}$ 

Оним из способов минимизации Q является градиентного спуск, алгоритм которого grad_descent($\alpha, \epsilon$, max_iter) в общем виде выглядит как:  
1. $W^{(0)} \leftarrow N(0,1)$ инициализировать начальное приближение для вектора весов небольшими случайными значениями, например из стандартного нормально распределения  
2. for i in range(max_iter):  
    2.1 $\nabla_wQ = [\frac{\partial Q}{\partial w_0}, \frac{\partial Q}{\partial w_1}, ..., \frac{\partial Q}{\partial w_d}]^T$ - посчитать градиент в точке $W = W^{(t)}$  
    2.2 $W^{(t+1)} = W^{(t)} - \alpha * \nabla_wQ$ - обновить вектор весов, сделав  шаг длины $\alpha$ в сторону антиградиента  
    2.3 Если $||\nabla_wQ ||_2 < \epsilon $  - если норма градиента стала меньше некоторой заданной малой величины, то мы считаем, что алгоритм сошелся и прерываем цикл
    
    
<b>Reading</b>:
1. http://www.machinelearning.ru/wiki/index.php?title=Simple_linear_regression
2. Bishop. Pattern Recognition and Machine Learning, 3.1
3. http://www.machinelearning.ru/wiki/index.php?title=Метод_градиентного_спуска

# Homework 5

Ваша задача - заполнить пропуски # YOUR CODE HERE, и выполнить код

В этой домашней работе необходимо реализовать одномерную линейную регрессию.  
Таким образом, в упрощенной постановке задачи d=1.

$f(x_i) = f(x_i; w) = w_1* x_i + w_0$   - наш алгоритм регрессии   
$Q(w) = \frac{1}{N} \sum_{i=1}^N (y_i - w_1 x_i - w_0)^2 \rightarrow \underset{w_1, w_0}{min}$ - наш MSE  

Градиент MSE по весам весам нашего алгоритма:  
$\nabla_wQ = [\frac{\partial Q}{\partial w_0}, \frac{\partial Q}{\partial w_1}]^T$

$\frac{\partial Q}{\partial w_0} = -\frac{2}{N} \sum_{i=1}^N (y_i - w_1 x_i - w_0)$  
$\frac{\partial Q}{\partial w_1} = -\frac{2}{N} \sum_{i=1}^N (y_i - w_1 x_i - w_0)x_i$  

Сам алгоритм градиентного спуска grad_descent($\alpha, \epsilon$, max_iter) будет выглядеть так:  

1. $w_1^{(0)}, w_0^{(0)} \leftarrow N(0,1)$ инициализировать начальное приближение для вектора весов небольшими случайными значениями, например из стандартного нормально распределения  
2. for i in range(max_iter):  
    2.1 $\nabla_wQ = [\frac{\partial Q}{\partial w_0}, \frac{\partial Q}{\partial w_1}]^T$ - посчитать градиент в точке $(w_1^{(t)}, w_0^{(t)})$  
    2.2 обновить вектор весов, сделав  шаг длины $\alpha$ в сторону антиградиента  
    $w_1^{(t+1)} = w_1^{(t)} - \alpha * \frac{\partial Q}{\partial w_1}$  
    $w_0^{(t+1)} = w_0^{(t)} - \alpha * \frac{\partial Q}{\partial w_0}$   
       
    2.3 Если $\sqrt{{\frac{\partial Q}{\partial w_0}}^2 + {\frac{\partial Q}{\partial w_1}}^2} < \epsilon $  - если норма градиента стала меньше некоторой заданной малой величины, то мы считаем, что алгоритм сошелся и прерываем цикл

In [3]:
import numpy as np

SEED = 17
np.random.seed(SEED)

In [30]:
class SimpleLinearRegression:
    def __init__(self, step = 0.01, tol = 1e-4, max_iter=1000, verbose=False, random_state=SEED):
        self.max_iter = max_iter # max iter count of gradient descent
        self.step = step # step of descent in the direction of antigradient
        self.tol = tol # we compare norm of gradient with that threshold
        self._w = None # w_1
        self._intercept = None # w_0
        self.random_state = random_state 
        self.verbose = verbose
        
    def predict(self, X):
        """
        estimate target variable "y" based on features X 
        """
        y_pred = self._w * X + self._intercept # MY CODE HERE
        assert y_pred.shape[0] == X.shape[0]
        return y_pred
    
    def score(self, X, y):
        """
        MSE
        X - features
        y - true values of target variable
        """
        return np.mean((y - self.predict(X))**2)
    
    def _gradient(self, X, y):
        """
        Compute gradient of MSE subject to w_1, w_0
        X - features
        y - true values of target variable
        """
        # MY CODE HERE
        grad_intercept = -(np.mean(y - self._w * X - self._intercept))
        grad_w = -(np.mean((y - self._w * X - self._intercept) * X))
        return grad_w, grad_intercept
        
    def fit(self, X, y):
        """
        Train model with gradient descent
        X - features
        y - true values of target variable
        """
        # for reproducable results
        np.random.seed(self.random_state)
        
        # initialize weights
        self._w, self._intercept = np.random.randn(2)
        # perform gradient descent
        for iter in range(self.max_iter):
            # compute gradient at current W
            grad_w, grad_intercept = self._gradient(X, y)
            
            # make step, update W
            # MY CODE HERE
            self._w = self._w - self.step * grad_w 
            self._intercept = self._intercept - self.step * grad_intercept 
            
            # compute gradient norm            
            grad_norm = np.sqrt(grad_w **2 + grad_intercept**2)# MY CODE HERE
            
            # people like to watch how the error is reducing during iterations 
            if self.verbose:
                mse_score = self.score(X, y)
                print('iteration %d, MSE = %f, ||grad|| = %f' % (iter, mse_score, grad_norm))
                
            # compare gradient norm with threshold
            if grad_norm < self.tol:
                print('model converged')
                return self
        print('model did not converge')
        return self

### Прикладная задача
Для Boston dataset мы хотим научится предсказывать значение target по признаку CRIM.  
При необходимости посмотрите прошлый семенар и документацию к pandas.

In [31]:
import pandas as pd
from sklearn import datasets


In [32]:
boston_data = datasets.load_boston()
boston_data.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR'])

In [33]:
df = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
df['target'] = boston_data['target']
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [34]:
print(boston_data['DESCR'])

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [35]:
# реализуем функцию, которая считает MSE 
def mse_score(y_true, y_pred):
    """
    y_true - true values of target variable
    y_pred - predicted values of target variable 
    """
    result = np.mean((y_true - y_pred)**2)
    return result

Разделим наш датасет на 2 части: на первой части мы будем обучаться, на второй оценивать качество работы алгоритма на новых данных

In [36]:
from sklearn.model_selection import train_test_split

# разбили датасет в соотношении 60:40
df_train, df_test = train_test_split(df, test_size=0.4, random_state=SEED, shuffle=True)

In [37]:
# обучите модель на df_train c verbose=True
# Обратите внимание на отладочный вывод, ваша ошибка MSE должна уменьшаться с каждой итерацией
# мы хотим научится предсказывать значение target по признаку CRIM


# MY CODE HERE
model = SimpleLinearRegression(verbose=True)
X_train = df_train['CRIM']
y_train = df_train['target']
model.fit(X_train, y_train)
mse_train_score = mse_score(y_train, model.predict(X_train))# MY CODE HERE

print('MSE on train:', mse_train_score)

iteration 0, MSE = 628.369508, ||grad|| = 43.878104
iteration 1, MSE = 617.535762, ||grad|| = 24.368769
iteration 2, MSE = 608.590002, ||grad|| = 21.309670
iteration 3, MSE = 599.959701, ||grad|| = 20.827030
iteration 4, MSE = 591.485412, ||grad|| = 20.627228
iteration 5, MSE = 583.149171, ||grad|| = 20.457445
iteration 6, MSE = 574.947211, ||grad|| = 20.291902
iteration 7, MSE = 566.877216, ||grad|| = 20.127986
iteration 8, MSE = 558.937048, ||grad|| = 19.965422
iteration 9, MSE = 551.124616, ||grad|| = 19.804175
iteration 10, MSE = 543.437867, ||grad|| = 19.644230
iteration 11, MSE = 535.874777, ||grad|| = 19.485577
iteration 12, MSE = 528.433358, ||grad|| = 19.328206
iteration 13, MSE = 521.111651, ||grad|| = 19.172105
iteration 14, MSE = 513.907732, ||grad|| = 19.017265
iteration 15, MSE = 506.819706, ||grad|| = 18.863676
iteration 16, MSE = 499.845707, ||grad|| = 18.711327
iteration 17, MSE = 492.983901, ||grad|| = 18.560208
iteration 18, MSE = 486.232485, ||grad|| = 18.410310
ite

iteration 234, MSE = 85.742843, ||grad|| = 3.194226
iteration 235, MSE = 85.542875, ||grad|| = 3.168428
iteration 236, MSE = 85.346124, ||grad|| = 3.142839
iteration 237, MSE = 85.152538, ||grad|| = 3.117457
iteration 238, MSE = 84.962067, ||grad|| = 3.092279
iteration 239, MSE = 84.774660, ||grad|| = 3.067305
iteration 240, MSE = 84.590267, ||grad|| = 3.042532
iteration 241, MSE = 84.408841, ||grad|| = 3.017960
iteration 242, MSE = 84.230334, ||grad|| = 2.993586
iteration 243, MSE = 84.054698, ||grad|| = 2.969409
iteration 244, MSE = 83.881888, ||grad|| = 2.945427
iteration 245, MSE = 83.711858, ||grad|| = 2.921639
iteration 246, MSE = 83.544563, ||grad|| = 2.898043
iteration 247, MSE = 83.379960, ||grad|| = 2.874637
iteration 248, MSE = 83.218004, ||grad|| = 2.851421
iteration 249, MSE = 83.058655, ||grad|| = 2.828392
iteration 250, MSE = 82.901868, ||grad|| = 2.805549
iteration 251, MSE = 82.747604, ||grad|| = 2.782890
iteration 252, MSE = 82.595822, ||grad|| = 2.760415
iteration 25

iteration 518, MSE = 73.436943, ||grad|| = 0.319295
iteration 519, MSE = 73.434945, ||grad|| = 0.316716
iteration 520, MSE = 73.432979, ||grad|| = 0.314158
iteration 521, MSE = 73.431045, ||grad|| = 0.311621
iteration 522, MSE = 73.429142, ||grad|| = 0.309104
iteration 523, MSE = 73.427269, ||grad|| = 0.306608
iteration 524, MSE = 73.425427, ||grad|| = 0.304132
iteration 525, MSE = 73.423614, ||grad|| = 0.301675
iteration 526, MSE = 73.421830, ||grad|| = 0.299239
iteration 527, MSE = 73.420075, ||grad|| = 0.296822
iteration 528, MSE = 73.418349, ||grad|| = 0.294425
iteration 529, MSE = 73.416650, ||grad|| = 0.292047
iteration 530, MSE = 73.414978, ||grad|| = 0.289688
iteration 531, MSE = 73.413333, ||grad|| = 0.287349
iteration 532, MSE = 73.411715, ||grad|| = 0.285028
iteration 533, MSE = 73.410123, ||grad|| = 0.282726
iteration 534, MSE = 73.408556, ||grad|| = 0.280443
iteration 535, MSE = 73.407015, ||grad|| = 0.278178
iteration 536, MSE = 73.405498, ||grad|| = 0.275931
iteration 53

iteration 713, MSE = 73.317997, ||grad|| = 0.065683
iteration 714, MSE = 73.317913, ||grad|| = 0.065153
iteration 715, MSE = 73.317830, ||grad|| = 0.064626
iteration 716, MSE = 73.317748, ||grad|| = 0.064104
iteration 717, MSE = 73.317667, ||grad|| = 0.063587
iteration 718, MSE = 73.317588, ||grad|| = 0.063073
iteration 719, MSE = 73.317510, ||grad|| = 0.062564
iteration 720, MSE = 73.317433, ||grad|| = 0.062058
iteration 721, MSE = 73.317358, ||grad|| = 0.061557
iteration 722, MSE = 73.317284, ||grad|| = 0.061060
iteration 723, MSE = 73.317211, ||grad|| = 0.060567
iteration 724, MSE = 73.317139, ||grad|| = 0.060078
iteration 725, MSE = 73.317068, ||grad|| = 0.059593
iteration 726, MSE = 73.316998, ||grad|| = 0.059111
iteration 727, MSE = 73.316930, ||grad|| = 0.058634
iteration 728, MSE = 73.316863, ||grad|| = 0.058160
iteration 729, MSE = 73.316796, ||grad|| = 0.057691
iteration 730, MSE = 73.316731, ||grad|| = 0.057225
iteration 731, MSE = 73.316667, ||grad|| = 0.056763
iteration 73

iteration 983, MSE = 73.312807, ||grad|| = 0.007355
iteration 984, MSE = 73.312806, ||grad|| = 0.007296
iteration 985, MSE = 73.312805, ||grad|| = 0.007237
iteration 986, MSE = 73.312804, ||grad|| = 0.007178
iteration 987, MSE = 73.312803, ||grad|| = 0.007120
iteration 988, MSE = 73.312802, ||grad|| = 0.007063
iteration 989, MSE = 73.312801, ||grad|| = 0.007006
iteration 990, MSE = 73.312800, ||grad|| = 0.006949
iteration 991, MSE = 73.312799, ||grad|| = 0.006893
iteration 992, MSE = 73.312799, ||grad|| = 0.006837
iteration 993, MSE = 73.312798, ||grad|| = 0.006782
iteration 994, MSE = 73.312797, ||grad|| = 0.006727
iteration 995, MSE = 73.312796, ||grad|| = 0.006673
iteration 996, MSE = 73.312795, ||grad|| = 0.006619
iteration 997, MSE = 73.312794, ||grad|| = 0.006566
iteration 998, MSE = 73.312793, ||grad|| = 0.006513
iteration 999, MSE = 73.312792, ||grad|| = 0.006460
model did not converge
MSE on train: 73.31279240364127


In [38]:
# # посчитали качество обученной модели на df_test

# MY CODE HERE
X_test = df_test['CRIM']
y_test = df_test['target']
mse_test_score = mse_score(y_test, model.predict(X_test))# MY CODE HERE

print('MSE on test:', mse_test_score)

MSE on test: 74.26585092144818
